In [ ]:
import gilp
import numpy as np
from ortools.linear_solver import pywraplp as OR

## Integer Program Examples

Consider the following 2D integer program:

$$\begin{align*}
\max \quad & 5x_1+8x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 6 \\
& 5x_1 + 9x_2 \leq 45 \\
& x_1, x_2 \geq 0, \quad \text{integer}
\end{align*}$$

In [ ]:
# Create the LP object
ip_2d = gilp.LP(A=[[1,1],
                   [5,9]],
                b=[6,45],
                c=[5,8])

In [ ]:
# Visualize branch and bound with manually selecting the branched on variable
gilp.bnb_visual(ip_2d, manual=True);

# Consider the following integer program:

$$\begin{align*}
\mbox{minimize } x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7\\
\mbox{subject to }\qquad 
x_1 + x_4 + x_5 + x_6 + x_7 & \ge 17 \\
x_1 + x_2 + x_5 + x_6 + x_7 & \ge 13 \\
x_1 + x_2 + x_3 + x_6 + x_7 & \ge 15 \\
x_1 + x_2 + x_3 + x_4 + x_7 & \ge 19 \\
x_1 + x_2 + x_3 + x_4 + x_5 & \ge 14 \\
x_2 + x_3 + x_4 + x_5 + x_6 & \ge 16 \\
x_3 + x_4 + x_5 + x_6 + x_7 & \ge 11 \\
x_i & \ge 0, \ \ i=1,\ldots,7, \mbox{ integer}
\end{align*}$$

In [ ]:
def Old_Work_Sched():
    # define model
    m = OR.Solver('ex', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i in range(1,8):
        x[i] = m.NumVar(0, m.infinity(), 'x_%d' % (i)) 
        
    # define objective function here
    m.Minimize(sum(x[i] for i in range(1,8)))
    
    # constraints
    m.Add(x[1]               + x[4] + x[5] + x[6] + x[7] >= 17)
    m.Add(x[1] + x[2]               + x[5] + x[6] + x[7] >= 13)
    m.Add(x[1] + x[2] + x[3]               + x[6] + x[7] >= 15)
    m.Add(x[1] + x[2] + x[3] + x[4]               + x[7] >= 19)
    m.Add(x[1] + x[2] + x[3] + x[4] + x[5]               >= 14)
    m.Add(       x[2] + x[3] + x[4] + x[5] + x[6]        >= 16)
    m.Add(              x[3] + x[4] + x[5] + x[6] + x[7] >= 11)
    
    return (m, x)  # return the model and the decision variables

In [ ]:
def solve(m):
    m.Solve()  
    print('Objective = %f' % (np.round(m.Objective().Value(),5))) 
    for var in m.variables():
        print(("%s: %f" % (var.name(), np.round(var.solution_value(),5))))

In [ ]:
m, x = Old_Work_Sched()
solve(m)

<img src="images-demo/old_work_sched/tree1.png" width="500">

In [ ]:
m, x = Old_Work_Sched()
m.Add(x[1] >= 2)
solve(m)

<img src="images-demo/old_work_sched/tree2.png" width="600">

In [ ]:
m, x = Old_Work_Sched()
m.Add(x[1] <= 1)
solve(m)

<img src="images-demo/old_work_sched/tree3.png" width="700">

In [ ]:
m, x = Old_Work_Sched()
m.Add(x[1] >= 2)
m.Add(x[2] <= 5)
solve(m)

<img src="images-demo/old_work_sched/tree4.png" width="700">

In [ ]:
m, x = Old_Work_Sched()
m.Add(x[1] >= 2)
m.Add(x[2] >= 6)
solve(m)

<img src="images-demo/old_work_sched/tree5.png" width="700">

In [ ]:
def Work_Sched(bonus = False):
    # define model
    m = OR.Solver('ex', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i in range(1,8):
        x[i] = m.NumVar(0, m.infinity(), 'x_%d' % (i)) 
        
    # objective function
    if not bonus:
        m.Minimize(36*x[1] + 40*x[2] + 36*x[3] + 32*x[4] + 32*x[5] + 32*x[6] + 32*x[7])
    else:
        m.Minimize(39.6*x[1] + 40*x[2] + 39.6*x[3] + 38.4*x[4] + 38.4*x[5] + 38.4*x[6] + 38.4*x[7])
    
    # constraints
    m.Add(x[1]               + x[4] + x[5] + x[6] + x[7] >= 3)  # Sunday
    m.Add(x[1] + x[2]               + x[5] + x[6] + x[7] >= 13) # Monday
    m.Add(x[1] + x[2] + x[3]               + x[6] + x[7] >= 14) # Tuesday
    m.Add(x[1] + x[2] + x[3] + x[4]               + x[7] >= 16) # Wednesday
    m.Add(x[1] + x[2] + x[3] + x[4] + x[5]               >= 17) # Thursday
    m.Add(       x[2] + x[3] + x[4] + x[5] + x[6]        >= 15) # Friday 
    m.Add(              x[3] + x[4] + x[5] + x[6] + x[7] >= 6)  # Saturday
        
    m.Add(45*x[1] + 54*x[2] + 51*x[3] + 44*x[4] + 44*x[5] + 48*x[6] + 44*x[7] >= 920)
    
    return (m, x)  # return the model and the decision variables

## Solve Without Weekend Bonus

In [ ]:
m, x = Work_Sched()
solve(m)

<img src="images-demo/work_sched_1/tree1.png" width="250">

In [ ]:
m, x = Work_Sched()
m.Add(x[2] >= 12)
solve(m)

<img src="images-demo/work_sched_1/tree2.png" width="250">

In [ ]:
m, x = Work_Sched()
m.Add(x[2] <= 11)
solve(m)

<img src="images-demo/work_sched_1/tree3.png" width="350">

In [ ]:
m, x = Work_Sched()
m.Add(x[2] <= 11)
m.Add(x[7] <= 0)
solve(m)

<img src="images-demo/work_sched_1/tree4.png" width="350">

In [ ]:
m, x = Work_Sched()
m.Add(x[2] <= 11)
m.Add(x[7] >= 1)
solve(m)

<img src="images-demo/work_sched_1/tree5.png" width="350">

## Solve With Weekend Bonus

In [ ]:
m, x = Work_Sched(True)
solve(m)

<img src="images-demo/work_sched_2/tree1.png" width="400">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] >= 1)
solve(m)

<img src="images-demo/work_sched_2/tree2.png" width="500">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] >= 1)
m.Add(x[2] >= 12)
solve(m)

<img src="images-demo/work_sched_2/tree3.png" width="500">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] >= 1)
m.Add(x[2] <= 11)
solve(m)

<img src="images-demo/work_sched_2/tree4.png" width="550">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] >= 1)
m.Add(x[2] <= 11)
m.Add(x[6] >= 1)
solve(m)

<img src="images-demo/work_sched_2/tree5.png" width="550">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] >= 1)
m.Add(x[2] <= 11)
m.Add(x[6] <= 0)
solve(m)

<img src="images-demo/work_sched_2/tree6.png" width="550">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] >= 1)
m.Add(x[2] <= 11)
m.Add(x[6] <= 0)
m.Add(x[5] >= 2)
solve(m)

<img src="images-demo/work_sched_2/tree7.png" width="550">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] >= 1)
m.Add(x[2] <= 11)
m.Add(x[6] <= 0)
m.Add(x[5] <= 1)
solve(m)

<img src="images-demo/work_sched_2/tree8.png" width="550">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] >= 1)
m.Add(x[2] <= 11)
m.Add(x[6] <= 0)
m.Add(x[5] <= 1)
m.Add(x[7] >= 1)
solve(m)

<img src="images-demo/work_sched_2/tree9.png" width="550">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] >= 1)
m.Add(x[2] <= 11)
m.Add(x[6] <= 0)
m.Add(x[5] <= 1)
m.Add(x[7] <= 0)
solve(m)

<img src="images-demo/work_sched_2/tree10.png" width="550">

In [ ]:
m, x = Work_Sched(True)
m.Add(x[5] <= 0)
solve(m)

<img src="images-demo/work_sched_2/tree11.png" width="550">